In [2]:
from typing import Annotated, Optional, TypedDict
from langgraph.graph import add_messages, StateGraph, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from uuid import uuid4
import json

In [3]:
load_dotenv()

True

In [4]:
llm = ChatOpenAI(model="gpt-4o")

In [5]:
search_tool = TavilySearch(max_results=4)

In [6]:
tools = [search_tool]

In [7]:
model_with_tools = llm.bind_tools(tools=tools)

In [8]:
checkpointer = MemorySaver()

In [9]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [10]:
async def model(state: State):
    response = await model_with_tools.ainvoke(state["messages"])
    return {"messages": [response]}

In [11]:
graph = StateGraph(State)

# Add nodes
graph.add_node("model", model)
graph.add_node("tool_node", ToolNode(tools=tools))


# Add Edges
graph.set_entry_point("model")
graph.add_conditional_edges(
    "model", tools_condition, {"tools": "tool_node", "__end__": END}
)
graph.add_edge("tool_node", "model")

workflow = graph.compile(checkpointer=checkpointer)

In [12]:
CONFIG = {"configurable": {"thread_id": 2}, "project_name": "my-graph-demo"}

response = await workflow.ainvoke(
    {"messages": [HumanMessage("When is the next ISRO lauch?")]}, config=CONFIG
)

In [13]:
response

{'messages': [HumanMessage(content='When is the next ISRO lauch?', additional_kwargs={}, response_metadata={}, id='07963f87-dfbc-4a55-a07d-42be234d6fe3'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Gh2dNuVC5Sm3aKpbmKk1rjVS', 'function': {'arguments': '{"query":"next ISRO launch date 2023"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 1277, 'total_tokens': 1300, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 1152}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C95KGPDz6GBeoXH8dyGai4yndAO8Q', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0e3b31b4-d49a-4a87-bf4c-838ea613d528-0', tool_calls=[{'name': 'tavily_search', 'args': {'qu

In [14]:
CONFIG = {"configurable": {"thread_id": 8}, "project_name": "my-graph-demo"}

async for event in workflow.astream_events(
    {"messages": [HumanMessage("When is the next ISRO lauch?")]}, config=CONFIG
):
    if event.get("event") == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)
        

The next ISRO launch is the "Space MAITRI (Mission for Australia-India’s Technology, Research and Innovation)" utilizing the Launch Vehicle Mark-3 (GSLV Mk III). The mission will be launched from the second launch pad at Satish Dhawan Space Centre. For more details, you can check the [ISRO Launch Schedule on RocketLaunch.org](https://rocketlaunch.org/launch-schedule/indian-space-research-organization).